In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!apt-get install unzip
!unzip 'filename.zip' -d "/content/drive/MyDrive/"
!unzip -FF "/content/gdrive/MyDrive/2_reviews_per_movie_raw.zip"


Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
unzip:  cannot find or open filename.zip, filename.zip.zip or filename.zip.ZIP.
Archive:  /content/gdrive/MyDrive/2_reviews_per_movie_raw.zip
   creating: 2_reviews_per_movie_raw/
  inflating: 2_reviews_per_movie_raw/The Other Boleyn Girl 2008.csv  
  inflating: 2_reviews_per_movie_raw/Casablanca 1942.csv  
  inflating: 2_reviews_per_movie_raw/The Terminal 2004.csv  
  inflating: 2_reviews_per_movie_raw/Harry Potter and the Deathly Hallows_ Part 1 2010.csv  
  inflating: 2_reviews_per_movie_raw/12 Angry Men 1957.csv  
  inflating: 2_reviews_per_movie_raw/The Assassination of Jesse James by the Coward Robert Ford 2007.csv  
  inflating: 2_reviews_per_movie_raw/Bloody Sunday 2002.csv  
  inflating: 2_reviews_per_movie_raw/Shooter 2007.csv  
  inflating: 2_reviews_per_movie_raw

In [ ]:
#Load the libraries
!pip install stanza
from bs4 import BeautifulSoup
import os
import glob
from itertools import chain
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.stem import WordNetLemmatizer
import re,string,unicodedata
import stanza
import numpy as np
import pandas as pd
import nltk
stan_nlp = stanza.Pipeline()
lemmatizer = WordNetLemmatizer()
stanza.download('en')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=695772b69d20f0dd70d5baf6d99f2288d95fb8510a2dfd4e69089d9d42a094a8
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def get_movies(movie_with_review_and_meta, fld_path, temp):
    file_names = glob.glob(fld_path)

    for file in file_names:

        path = file.split('/')[-1]
        # title = path.rsplit(' ', 1)[0]
        print(path)
        if path not in movie_with_review_and_meta:
            continue

        # opening the file in read mode
        my_file = open('/content/gdrive/MyDrive/already_aspect_extract_movies.txt', "r")

        # reading the file
        already_extracted = my_file.read()

        # replacing end splitting the text
        # when newline ('\n') is seen.
        already_extracted_list = already_extracted.split("\n")
        my_file.close()
        if path in already_extracted_list:
            continue


        review_pre = pd.read_csv(file)
        #Apply function on review column
        test_review = review_pre['review']
        for rev in range(len(test_review)):
            test_review[rev] = denoise_text(test_review[rev])
            test_review[rev] = replace(test_review[rev])
            test_review[rev] = split_review(test_review[rev])
        oa_pairs_all = defaultdict(list)
        index = 0
        for review in test_review:
            if index==520:
              break
            for sentence in review:
                past_oa_pairs = {}
                doc = stan_nlp(" ".join(sentence.split()).lower())
                oa_pairs = {}
                parsing_result = {}
                for key, value in chain(past_oa_pairs.items(), parsing(doc, oa_pairs, parsing_result).items()):
                    key = lemmatizer.lemmatize(key)
                    if not wn.synsets(key):
                        continue
                    oa_pairs_all[key].append(value)
                    past_oa_pairs = oa_pairs_all
            index+=1
        with open('/content/gdrive/MyDrive/already_aspect_extract_movies.txt', 'a') as file:
          file.write('{}\n'.format(path))
        with open('/content/gdrive/MyDrive/movies_oa_pairs_2/'+ path, 'w', encoding="utf-8") as f:
          for k, v in oa_pairs_all.items():
            v = re.sub("\'|\[|\]", '', str(v)).lstrip().lower()
            f.write("%s, %s\n" % (k.lower(), v.lstrip()))

# reviews preprocessing
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    punctuation = '\"|\@|\*|\$|\%|\&|\^|\||\~|\_|\d|\t|\\\|\\/'
    return re.sub(punctuation, '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace(text):
    s = text
    for (pattern, repl) in patterns:
        (s, count) = re.subn(pattern, repl, s)
    return s
# remove repeated chac

def split_review(text):
    text = re.split(r'[,.?!--()]',text)
    while '' in text:
        text.remove('')
    return text

def check_nn(aspect_word,opinion_word):
    for nn in parsing_result[aspect_word]:
        if nn[1] == "compound":
            oa_pairs[nn[0] + " " + aspect_word] = opinion_word
            del oa_pairs[aspect_word]
            return nn[0] + " " + aspect_word
    return  aspect_word


def check_advmod(aspect_word,opinion_word):
    # print(aspect_word+" "+opinion_word)
    s_advmod = ""
    for advmod in parsing_result[opinion_word]:
        if advmod[1] == "advmod":
            s_advmod += advmod[0] + " "
            oa_pairs[aspect_word] = s_advmod + opinion_word

def parsing(doc, oa_pairs, parsing_result):
    multi_re = ['amod','xcomp','nsubj','cop','advmod','nsubj:pass','advmod','compound','aux','det']
    for sent in doc.sentences:
        for word in sent.words:
            if word.head > 0:
                opinion = sent.words[word.head-1].text
            else:
                opinion = "root"
            if word.deprel in multi_re :
                    try:
                        parsing_result[opinion].append([word.text, word.deprel])
                    except:
                        parsing_result[opinion] = [[word.text, word.deprel]]
        for  i in parsing_result.keys():
            for j in parsing_result[i]:
                if j[1] in ['amod']:
                    oa_pairs[i] = j[0]
                    for f_neg in parsing_result[i]:
                        if f_neg[1] == "det" and f_neg[0] == "no":
                            oa_pairs[i] = "no " + oa_pairs[i]
                            try:
                                check_advmod(i, j[0])
                            except:
                                pass
                elif j[1] in ['dobj','acomp','xcomp']:
                    for k in parsing_result[i]:
                        if k[1] == "nsubj":
                            oa_pairs[k[0]] = j[0]
                            try:
                                check_advmod(k[0], j[0])
                            except:
                                pass

                elif j[1] == 'advmod':
                    for k in parsing_result[i]:
                        if k[1] == "nsubj:pass":
                            oa_pairs[k[0]] = j[0]
                            try:
                                check_advmod(k[0], j[0])
                            except:
                                pass
                elif j[1] == 'cop':
                    for k in parsing_result[i]:
                        if k[1] == "nsubj":
                            oa_pairs[k[0]] = i
                            try:
                                check_advmod(k[0], oa_pairs[k[0]])
                            except:
                                pass

                            # # ----------------check_neg_through_hypo()-----------------
                            aux_stack = ""
                            for mod in parsing_result[i]:
                                if mod[1] == 'aux':
                                    aux_stack += mod[0] + " "
                            oa_pairs[k[0]] = aux_stack + oa_pairs[k[0]]
                        # ---------------------------------------------------------

    # print(parsing_result)
    # print(oa_pairs)
    return oa_pairs


!pip install pickle
import pickle
global replacement_patterns, patterns, repeat_reg, repl
replacement_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would')]
patterns = [(re.compile(regex), repl) for (regex, repl) in replacement_patterns]
repeat_reg = re.compile(r'(\w*)(\w)\2(\w*)')
repl = r'\1\2\3'

# with open("/content/gdrive/MyDrive/fin_movies_meta_wr_a1994_titles", "rb") as fp:   # Unpickling
#   movie_with_review_and_meta = pickle.load(fp)
with open("/content/gdrive/MyDrive/for_extract_2", "rb") as fp:   # Unpickling
  movie_with_review_and_meta = pickle.load(fp)
# print(movie_with_review_and_meta)
# print(len(movie_with_review_and_meta))

path = '/content/2_reviews_per_movie_raw/*.csv'
get_movies(movie_with_review_and_meta, fld_path = path, temp = True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
The Devil Wears Prada 2006.csv
The Twilight Saga_ New Moon 2009.csv
Logan 2017.csv
Yesterday 2019.csv
To Kill a Mockingbird 1962.csv
Batman v Superman_ Dawn of Justice 2016.csv
The Patriot 2000.csv
Braveheart 1995.csv
Underworld 2003.csv
Fantastic 4_ Rise of the Silver Surfer 2007.csv
Cars 2 2011.csv
The Evil Dead 1981.csv
Cars 2006.csv
Back to the Future Part III 1990.csv
Dog Day Afternoon 1975.csv
Knocked Up 2007.csv
World Trade Center 2006.csv
Just Go with It 2011.csv
Never Back Down 2008.csv
The Fifth Element 1997.csv
Training Day 2001.csv
Unbreakable 2000.csv
30 Days of Night 2007.csv
Wanted 2008.csv
Midnight in Paris 2011.csv
Terminator 3_ Rise of the Machines 2003.csv
The Secret Life of Pets 2016.csv
The Sixth Sense 1999.csv
John Wick 2014.csv

<ipython-input-4-4347207d9607>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = denoise_text(test_review[rev])
<ipython-input-4-4347207d9607>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = replace(test_review[rev])
<ipython-input-4-4347207d9607>:84: FutureWarning: Possible set difference at position 5
  text = re.split(r'[,.?!--()]',text)
<ipython-input-4-4347207d9607>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

Saw III 2006.csv
Stalag 17 1953.csv
It Follows 2014.csv
The Notebook 2004.csv
Corpse Bride 2005.csv
The Hangover 2009.csv
The Hangover Part II 2011.csv
Cloud Atlas 2012.csv
Bridge of Spies 2015.csv
Rogue One_ A Star Wars Story 2016.csv
Harry Potter and the Prisoner of Azkaban 2004.csv
Scream 1996.csv
Life 2017.csv
Million Dollar Baby 2004.csv
How to Lose a Guy in 10 Days 2003.csv
Dark City 1998.csv
Sing Street 2016.csv
The Nice Guys 2016.csv
The Book Thief 2013.csv
La Vie En Rose 2007.csv
Harry Potter and the Order of the Phoenix 2007.csv
Fargo 1996.csv
BASEketball 1998.csv
Finding Neverland 2004.csv
Ray 2004.csv
Howl's Moving Castle 2004.csv
One Flew Over the Cuckoo's Nest 1975.csv
Scream 2 1997.csv
22 Jump Street 2014.csv
Duck Soup 1933.csv
Watchmen 2009.csv
Halloween 1978.csv
Across the Universe 2007.csv
Nausicaä of the Valley of the Wind 1984.csv
Men in Black 1997.csv
Atonement 2007.csv
Love Actually 2003.csv
Match Point 2005.csv
Source Code 2011.csv
The Girl Next Door 2004.csv
Ali

<ipython-input-4-4347207d9607>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = denoise_text(test_review[rev])
<ipython-input-4-4347207d9607>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = replace(test_review[rev])
<ipython-input-4-4347207d9607>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = split_review(test_review[rev])


Bram Stoker's Dracula 1992.csv
Transformers_ Dark of the Moon 2011.csv
Tucker and Dale vs Evil 2010.csv
Brokeback Mountain 2005.csv
Beasts of No Nation 2015.csv
Iron Man 2008.csv
Amistad 1997.csv
Casualties of War 1989.csv
Hustle & Flow 2005.csv
Con Air 1997.csv
The Caine Mutiny 1954.csv
Wedding Crashers 2005.csv
The Lobster 2015.csv
3 Idiots 2009.csv
Pi 1998.csv
Oblivion 2013.csv
Stalingrad 1993.csv
The Replacements 2000.csv
The Impossible 2012.csv
Megamind 2010.csv
Spotlight 2015.csv
Vanilla Sky 2001.csv
Stop-Loss 2008.csv
The Ottoman Lieutenant 2017.csv
For Love of the Game 1999.csv
Predator 1987.csv
Alien vs. Predator 2004.csv
The Holiday 2006.csv
Lincoln 2012.csv
Macbeth 2015.csv
Shrek 2 2004.csv
The Rocker 2008.csv
Carrie 1976.csv
American Hustle 2013.csv
Mr. & Mrs. Smith 2005.csv
Transformers 2007.csv
Marriage Story 2019.csv
The Purge 2013.csv
Foxcatcher 2014.csv
The Producers 1967.csv
Deepwater Horizon 2016.csv
The Alamo 2004.csv
The Insider 1999.csv
The Thin Red Line 1998.csv


...

Evil Dead II 1987.csv
Bridesmaids 2011.csv
Dallas Buyers Club 2013.csv
Fantastic Mr. Fox 2009.csv
Star Wars_ Episode V - The Empire Strikes Back 1980.csv
Rab Ne Bana Di Jodi 2008.csv
Tenacious D in the Pick of Destiny 2006.csv
The Hunger Games 2012.csv
Star Trek Into Darkness 2013.csv
Die Hard 1988.csv
Beauty and the Beast 2017.csv
The Shawshank Redemption 1994.csv


<ipython-input-4-4347207d9607>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = denoise_text(test_review[rev])
<ipython-input-4-4347207d9607>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = replace(test_review[rev])
<ipython-input-4-4347207d9607>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = split_review(test_review[rev])
<ipython-input-4-4347207d9607>:61: MarkupResemblesL

About Time 2013.csv
The Town 2010.csv
Kagemusha 1980.csv
Billy Elliot 2000.csv
Ocean's Eleven 2001.csv
Zodiac 2007.csv
The Favourite 2018.csv
There Will Be Blood 2007.csv


<ipython-input-4-4347207d9607>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = denoise_text(test_review[rev])
<ipython-input-4-4347207d9607>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = replace(test_review[rev])
<ipython-input-4-4347207d9607>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_review[rev] = split_review(test_review[rev])
<ipython-input-4-4347207d9607>:61: MarkupResemblesL

Rocky IV 1985.csv
Zootopia 2016.csv
Coriolanus 2011.csv
Centurion 2010.csv
The Mummy 1999.csv
The Perks of Being a Wallflower 2012.csv
Reservoir Dogs 1992.csv
Gremlins 1984.csv
The Lord of the Rings_ The Two Towers 2002.csv
Home Alone 2_ Lost in New York 1992.csv
True Romance 1993.csv
Troy 2004.csv
The Last Song 2010.csv
American Psycho 2000.csv
Captain Phillips 2013.csv
Resident Evil_ Afterlife 2010.csv
Pirates of the Caribbean_ The Curse of the Black Pearl 2003.csv
Step Up 2_ The Streets 2008.csv
Grudge Match 2013.csv
The Lives of Others 2006.csv
Music and Lyrics 2007.csv
X-Men_ The Last Stand 2006.csv
Despicable Me 2010.csv
Departures 2008.csv
Jackie Brown 1997.csv
Grindhouse 2007.csv
The Breakfast Club 1985.csv
Quiz Show 1994.csv
Friday Night Lights 2004.csv
The Benchwarmers 2006.csv
Battleship Potemkin 1925.csv
Fast & Furious 6 2013.csv
Pitch Perfect 3 2017.csv
The Irishman 2019.csv
Ratatouille 2007.csv
The Ring 2002.csv
Jerry Maguire 1996.csv
The Conjuring 2 2016.csv
The Double L